# Summary of the notation of the paper and its translation into code

## Constraints
Arc $a=(i,j)$ is closed for all other trains when a train travels along it from station $i$ to statoin $j$, departing at time $t$ and traveling for duration $d(a)$ . This closing period is denoted as

$$[t, t+d(a)]$$

A station is closed for a security duration $\Delta$ after a train leaves a 
station.

A station has separate capacities for parking and passing through. Only one train can be parked at a station at each moment, but on other train can pass through the station while one other train is parked there.

## Sets
$G=(V, A)$ is the graph, where $V=\{1, 2, \dots, N\}$. Arcs are directional, where arc $(i,j)$ leads from node $i$ to node $j$ (and vice versa). All arcs can be travelled in both directions.

The journey of one train, its path, is denoted as $P=\{1, i_1, i_2, \dots, N\}$, so that $1$ is the origin of the path, $N$ its destination, and $i_1, i_2, \dots$ are the nodes travelled. A feasible path is one where all movement constraints and time constraints are fulfilled.


## Splitting the problem

When a train arrives at a node $i$ at time $t$, there are two possibilities:

1. Parking is permitted for some time period.
2. Parking is not permitted, and it is permitted to continue along an arc.

If these two conditions are not fulfilled, then the path is infeasible at this node. In this problem, it is assumed that some feasible path exists.

The time at node $1$ is zero.
Parking at node $i$ is permitted in the interval $[\alpha_i, \beta_i]$. The time period $p_i$ denotes the time during which a train parks at node $i$.

Departure from node $i$ to node $j$ is allowed in the time interval $[\gamma_{ij}, \delta_{ij}]$, and travel time on arc $(i,j)$ is $t_{ij}$. The following holds:

\begin{align*}
\beta_i > \alpha_i > 0 &&  \gamma_{ij} > \delta_{ij} > 0 \\
p_i \leq \beta_i - \alpha_i
\end{align*}


### Definition of a feasible arrival time
A feasible arrival time is defined as the time of arrical at node $i$ where a train can park for a period $p_i$ and then depart along $(i,j)$.
$f(i,j)$ is the earliest feasible arrival time at node $i$, provided that one can depart from $i$ along $(i,j)$.

For nodes $i \neq j \neq k$ and non-blocked routes $(i,j)$ and $(j,k)$, the earliest arrival time $f(j,k) = \underset{i, p_i}{\min}[f(i,j) + p_i + t_{ij}]$; that is, the earliest feasible arrival time at node $j$ (departing to node $k$) is the sum of the earliest arrival time at node $i$ departing to node $j$, waiting time at node $i$, and travel time on arc $(i,j)$, for the node $i$ and waiting time $p_i$ minimising that sum.

For blocked routes $(i,j)$, we may be able to increase parking time at some previous node to increase $f(i,j)$ such that $f(i,j) = \gamma_{ij}$ or $f(i,j) = \alpha_i$ to unblock arc $(i,j)$. This implies that there may be nodes at which $f(i_1, i_2)$ takes on different values depending on what route it is part of. The authors claim that this does normally not occur.

### Blocked route

A route along arc $(i,j)$ is blocked if one of these conditions is fulfilled:

1. Departure is allowed before parking (it is not necessary to wait for departure while parking), but arrival occurs before departure is allowed: $f(i,j) < \gamma_{ij} < \alpha_i$
2. Departure is allowed at some point in time during the parking period (it is possible to wait for departure while parking), but arrival occurs before parking is allowed: $f(i,j) < \alpha_i < \gamma_{ij} < \beta_i$
3. Departure is allowed after the parking period (it is not possible to wait for departure while parking), and arrival occurs before parking: $f(i,j) < \alpha_i < \beta_i < \gamma_{ij}$
4. Arrival occurs after departure is allowed: $\delta_{ij} < f(i,j)$

## Solution procedure
The main part of the solution must be to calculate the feasible arrival times at each part of some route.

For non-blocked arcs $(i,j)$ and $(j,k)$, the feasible arrival time at node $j$ departing to node $k$ $f(j,k)$ can be directly calculated form the arrival time at the previous node, along with the waiting time at the previous node and the travel time.

Blocked routes where $\delta_{ij} < f(i,j)$ can not be unblocked; here, $f(i,j) = \infty$.

For blocked routes that are blocked due to one of the conditions 1-3, we can try to unblock these routes. We do so 
